In [11]:
import os
import numpy as np
import cv2
import torch
from torch import nn, optim
from torch.autograd import Variable
from torchvision import transforms
import keras
import numpy as np
from keras import backend as K
from keras.models import model_from_json
import json
from google.colab.patches import cv2_imshow
from PIL import Image

In [12]:
%cd '/content/drive/MyDrive/Colab Notebooks/football'

/content/drive/MyDrive/Colab Notebooks/football


In [13]:
# %cd /content
# !mkdir croped_image
!cp '/content/drive/MyDrive/Colab Notebooks/football/y2mate.com - Real Madrid vs FC Barcelona 4K50fps 21112015 04 La Liga_1080pFHR_1.mp4' '/content'

In [14]:
# %cd /content
# !mkdir croped_image
# cap = cv2.VideoCapture('/content/y2mate.com - Real Madrid vs FC Barcelona 4K50fps 21112015 04 La Liga_1080pFHR_1.mp4')
# Width = int(cap.get(3))
# Height = int(cap.get(4))
# # out_vid = cv2.VideoWriter('./final_output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), cap.get(cv2.CAP_PROP_FPS), (Width,Height))
# if (cap.isOpened()== False):
#   print("Error opening video stream or file")
# scale = 0.00392
# frame_count = 0
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# # Read until video is completed
# while(cap.isOpened()):
#     # Capture frame-by-frame
#     ret, image = cap.read()
#     if not ret:
#       break
#     if frame_count%10==0:
#         blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
#         net.setInput(blob)
#         outs = net.forward(get_output_layers(net))
#         class_ids = []
#         confidences = []
#         boxes = []
#         conf_threshold = 0.5
#         nms_threshold = 0.4
#         for out in outs:
#             for detection in out:
#                 scores = detection[5:]
#                 class_id = np.argmax(scores)
#                 if class_id == 0:
#                     confidence = scores[class_id]
#                     if confidence > 0.5:
#                         center_x = int(detection[0] * Width)
#                         center_y = int(detection[1] * Height)
#                         w = int(detection[2] * Width)
#                         h = int(detection[3] * Height)
#                         x = center_x - w / 2
#                         y = center_y - h / 2
#                         class_ids.append(class_id)
#                         confidences.append(float(confidence))
#                         boxes.append([x, y, w, h])
#             indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
#         player_num = 0
#         for i in indices:
#             if player_num%3==0:
#                 player_num+=1
#                 i = i[0]
#                 box = boxes[i]
#                 x = max(0,box[0])
#                 y = max(0,box[1])
#                 w = box[2]
#                 h = box[3]
#                 detected_player = image[round(y):round(y+h), round(x):round(x+w)]
#                 cv2.imwrite(f'croped_image/frame_{frame_count}_player_{player_num}.jpg',detected_player)
#             player_num+=1
#             # if w>5 and h>5 and x>=0 and y>=0:
#             #     team = predict_image(detected_player.copy())
#             #     draw_prediction(image, team, confidences[i], round(x), round(y), round(x+w), round(y+h))
#             # else:
#             #   print(x,y,w,h)
#         # # show ouput frame by frame
#         # cv2_imshow(image)
#         # out_vid.write(image)
#     if frame_count%10==0:
#         print(f'frames done {frame_count}/{total_frames}')
#     frame_count+=1

# cap.release()
# # out_vid.release()

In [15]:
# !ls
# !zip -r dataset2.zip croped_image/

In [16]:
!ls '/content/drive/MyDrive/Colab Notebooks/football/train'

bar  crowd  ref  rma


In [17]:
classes = ['bar','crowd','ref','rma']
classes.sort()
print(classes)

['bar', 'crowd', 'ref', 'rma']


In [18]:
# download pretrained yolo weights
# !wget https://pjreddie.com/media/files/yolov3.weights

In [19]:
# def validInt(num):
#   try:
#     int(num)
#     return True
#   except:
#     return False

In [20]:
classes = ['FCB', 'crowd', 'ref', 'RMA']

def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers


def draw_prediction(img, id, confidence, x, y, x_plus_w, y_plus_h, player_id=None):
    if id==0:
      label = 'FCB'
      color = (0, 0, 255)
    elif id==1:
      return
    elif id == 2:
      label = 'ref'
      color = (255,215,0)
    elif id == 3:
      label = 'RMA'
      color = (255,255,255)
    else:
      label = 'ball'
      color = (0,0,0)
    if player_if is not None and validInt(player_num):
      label += f' {player_num}'
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [21]:
%cd '/content/drive/MyDrive/Colab Notebooks/football'
net = cv2.dnn.readNet('./yolov3.weights', './yolov3.cfg')

/content/drive/MyDrive/Colab Notebooks/football


In [22]:
import math
def calculate_avg_color(image):
  shape = image.shape
  print(shape)
  avg_color = [0,0,0]
  num_pix = 0
  for x in range(shape[0]):
    for y in range(shape[1]):
      r,b,g = image[x][y] 
      if not (r <5 and b <5 and g<5):
        avg_color+=image[x][y]
        num_pix += 1
  avg_color = avg_color/num_pix
  print('dist',math.sqrt(avg_color[0]**2+avg_color[1]**2+avg_color[2]**2)/3)
  return avg_color

In [29]:
%cd '/content/drive/MyDrive/Colab Notebooks/football'
# defining the model architecture
class Net(nn.Module):   
  def __init__(self):
      super(Net, self).__init__()

      self.cnn_layers = nn.Sequential(
          # Defining a 2D convolution layer
          nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          # Defining another 2D convolution layer
          nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
      )

      self.linear_layers = nn.Sequential(
          nn.Linear(4 * 8 * 8, 4)
      )

  # Defining the forward pass    
  def forward(self, x):
      x = self.cnn_layers(x)
      x = x.view(x.size(0), -1)
      x = self.linear_layers(x)
      return x

model = Net()
# load pretrained model
model.load_state_dict(torch.load('./teamCategoryModel2.pth'))
model.eval()
model.cuda()

/content/drive/MyDrive/Colab Notebooks/football


Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=256, out_features=4, bias=True)
  )
)

In [30]:
# transformations to be applied on images
image_transform = transforms.Compose([
                                transforms.Resize((32,32)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                              ])

def predict_image(image):
    # You may need to convert the color.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image_tensor = image_transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.cuda()
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [31]:
def decode_nn_res(res_vec, num_digits, num_classes, dummy_class):
    digits = np.array_split(res_vec, num_digits)
    actual_digits = np.argmax(digits,1)+1
    res = actual_digits[actual_digits!=dummy_class]
    return actual_digits, ''.join(map(str, res))

def process_labels(labels,max_digits):
    tmp = []
    for label in labels:
        vec = [int(float(x)) for x in label.split('_')]
        if len(vec) < max_digits:
            vec = vec + [11]*(max_digits-len(vec))
        tmp.append(vec)
    labels = np.array(tmp)
    tmp = []
    num_classes = 11
    for target in labels[:,...]:
        y = np.zeros((len(target), num_classes))
        y[np.arange(target.shape[0]), target-1] = 1
        tmp.append(y)
    labels = np.array(tmp)   
    return labels

In [32]:
# load model for player number detection
base_path = '/content/drive/MyDrive/Colab Notebooks/football/model_weights'
#load digit detection model
with open('./model_weights/digit_detection_cnn_layers.json','r') as json_data:
    model_dict = json.load(json_data)

detect_model = model_from_json(json.dumps(model_dict))
detect_model.load_weights('./model_weights/digit_detection_cnn_weights.h5')

#load digit classification model
with open('./model_weights/digit_classification_cnn_layers.json','r') as json_data:
    model_dict = json.load(json_data)

classification_model = keras.models.model_from_json(json.dumps(model_dict))
classification_model.load_weights(os.path.join('./model_weights/digit_classification_cnn_weights.h5'))

In [33]:
def find_box_and_predict_digit(input_img):
    num_digits = 4
    input_img_shape = input_img.shape
    train_img_size = (64,64)
    proc_input_img = np.array(cv2.normalize(cv2.cvtColor(cv2.resize(input_img,train_img_size), cv2.COLOR_BGR2GRAY).astype(np.float64), 0, 1, cv2.NORM_MINMAX)[...,np.newaxis])[np.newaxis,...]
    box_preds = detect_model.predict(proc_input_img)
    scaled_box = box_preds[0].copy()
    scaled_box[0] = scaled_box[0]/float(train_img_size[0]/input_img_shape[0])
    scaled_box[1] = scaled_box[1]/float(train_img_size[1]/input_img_shape[1])
    scaled_box[2] = scaled_box[2]/float(train_img_size[1]/input_img_shape[1])
    scaled_box[3] = scaled_box[3]/float(train_img_size[0]/input_img_shape[0])
    start_row = np.clip(int(scaled_box[0]),1,input_img_shape[0])
    end_row = np.clip(int(scaled_box[0]+scaled_box[3]),1,input_img_shape[0])
    start_col = np.clip(int(scaled_box[1]),1,input_img_shape[1])
    end_col = np.clip(int(scaled_box[1]+scaled_box[2]),1,input_img_shape[1])
    #need better logic to handle cases where the box is too thin
    if start_col-end_col==0:
        start_col -=1
    if start_row-end_row==0:
        start_row -=1
    #store only the cutouts
    digits_only = input_img[start_row:end_row,start_col:end_col,...]
    digits_only_resized = cv2.resize(digits_only,train_img_size)
    orig_img_box = input_img.copy()
    # cv2.rectangle(orig_img_box,(start_col,start_row),(end_col,end_row),(0,255,0),1)
    # plt.imshow(orig_img_box)
    # plt.show()
    digit_pred = classification_model.predict(np.array(digits_only_resized)[np.newaxis,...])
    score = np.concatenate(digit_pred, axis=1)
    pred_labels_encoded = np.zeros(score.shape, dtype="int32")
    pred_labels_encoded[score > 0.5] = 1
    pred_labels_decoded = np.array([decode_nn_res(x,num_digits,11,11) for x in pred_labels_encoded])
    pred_labels_decoded_digits = np.array(pred_labels_decoded[:,1])
    #pred_labels_decoded_OHE_digits = np.array(pred_labels_decoded[:,0])
    final_digit = pred_labels_decoded_digits[0]
    print('Predicted digit:',final_digit)
    return final_digit

In [ ]:
cap = cv2.VideoCapture('./y2mate.com - Real Madrid vs FC Barcelona 4K50fps 21112015 04 La Liga_1080pFHR_1.mp4')
Width = int(cap.get(3))
Height = int(cap.get(4))
out_vid = cv2.VideoWriter('./final_output2.avi',cv2.VideoWriter_fourcc('M','J','P','G'), cap.get(cv2.CAP_PROP_FPS), (Width,Height))
if (cap.isOpened()== False):
  print("Error opening video stream or file")
scale = 0.00392
frame_count = 0
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, image = cap.read()
    if not ret:
      break
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(get_output_layers(net))
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            if class_id == 0 or class_id==32:
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])
                    class_ids.append(class_id)
        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    player_num = 0
    for num,i in enumerate(indices):
        player_num+=1
        i = i[0]
        box = boxes[i]
        x = max(0,box[0])
        y = max(0,box[1])
        w = box[2]
        h = box[3]
        detected_player = image[round(y):round(y+h), round(x):round(x+w)]
        if class_ids[num]==0:
          if w>5 and h>5 and x>=0 and y>=0:
              id = predict_image(detected_player.copy())
              player_id = find_box_and_predict_digit(detected_player)
              draw_prediction(image, id, confidences[i], round(x), round(y), round(x+w), round(y+h), player_id)
          else:
            print(x,y,w,h)
        else:
          id=5
          draw_prediction(image, id, confidences[i], round(x), round(y), round(x+w), round(y+h))
    # # show ouput frame by frame
    cv2_imshow(image)
    out_vid.write(image)
    if frame_count%10==0:
        print(f'frames done {frame_count}/{total_frames}')
    frame_count+=1
    if frame_count>2:
      break

cap.release()
out_vid.release()